In [ ]:
import sys#THIS IS FOR GOOGLE COLAB USERS ONLY
from google.colab import files
sys.path.insert(0,'/content/drive/MyDrive/IR-Project-Colab')

In [ ]:
!cp /content/drive/MyDrive/IR-Project-Colab/Utilities.py /content #THIS IS FOR GOOGLE COLAB USERS ONLY
!cp /content/drive/MyDrive/IR-Project-Colab/KNNImpute.py /content

In [19]:
# Importing Libraries

import numpy as np
import pandas as pd


import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#------ Additional Lib
import neattext as nt
import neattext.functions as nfx

from sklearn.feature_extraction.text import TfidfVectorizer   # Turning textual data into numeric for computation
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder               # For encoding categorical target attr
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

from sklearn.tree import DecisionTreeClassifier
from sklearn import svm   # Baseline
from sklearn.svm import SVC

# ------- Validation metrics
from sklearn.metrics import accuracy_score    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss                  
from sklearn.metrics import classification_report



In [1]:
# import Utilities  #THIS IS FOR GOOGLE COLAB USERS ONLY
# import KNNImpute

from Utilities import *
from KNNImpute import *


In [58]:
import nltk
import numpy as np
nltk.download('stopwords')
import pandas as pd
labels=["country", "sku_id", "title", "category_lvl1","category_lvl2","category_lvl3", "description", "price", "type"]

def getTrainingDataset():
    return pd.read_csv('data_train.csv', header=None, names=labels)  


def NullStatistics(df):
    missing_val = df.isnull().sum()
    print(missing_val)
    total_cells = np.product(df.shape)
    missing_percent = (missing_val.sum()/total_cells) *100
    print(f'\nThe missing data percent is: {missing_percent}')

trainingDataset=getTrainingDataset()

# train_df,Y1,Y2,Y3=Utilities.Cleaning_Data_Utility(trainingDataset) #Utilities. hatadena yahan se  COLAB
train_df,Y1,Y2,Y3=Cleaning_Data_Utility(trainingDataset)   #NOTEBOOK USERS
# print(train_df)

NullStatistics(train_df)

# unique_label_c1, unique_label_c2, unique_label_c3 = KNNImpute.preserve_label(train_df) #It has all unique values lying in ctg1 , 2 , 3 column
unique_label_c1, unique_label_c2, unique_label_c3 = preserve_label(train_df)   #notebook

# KNNImpute.encode(['category_lvl1', 'category_lvl2', 'category_lvl3'],train_df)    #Performed encoding for CTGLVL3 KNN #COLAB
encode(['category_lvl1', 'category_lvl2', 'category_lvl3'],train_df)    #NOTEBOOK

# train_df_imputed = KNNImpute.impute(train_df)  #COLAB
train_df_imputed = impute(train_df)            #NOTEBOOK

# train_df_imputed = KNNImpute.clean_csv(train_df_imputed,train_df)   #COLAB
train_df_imputed = clean_csv(train_df_imputed,train_df)             #NOTEBOOK

NullStatistics(train_df_imputed)

[nltk_data] Downloading package stopwords to C:\Users\Computer
[nltk_data]     Point\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        Adana Gallery Suri Square Hijab – Light Pink <...
1        Cuba Heartbreaker Eau De Parfum Spray 100ml/3....
2        Andoer 150cm Cellphone Smartphone Mini Dual-He...
3        ANMYNA Complaint Silky Set 柔顺洗发配套 (Shampoo 520...
4        Argital Argiltubo Green Clay For Face and Body...
                               ...                        
36278    SADES K10 LED Backlit Wired USB Mechanical Gam...
36279    SONA 20L Electric Oven SEO 2220 <ul> <li>&nbsp...
36280    OP1001 Portable Wireless Bluetooth 2.1 Speaker...
36281    Woot-Woot TicTacToe Pillow Case (White) <ul> <...
36282    New Smart Wristband D21 Smart Bracelet Heart R...
Name: titleDescp, Length: 36283, dtype: object
category_lvl1       0
category_lvl2       0
category_lvl3    2135
titleDescp          0
dtype: int64

The missing data percent is: 1.4710746079431138


C:\Users\Computer Point\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


category_lvl1    0
category_lvl2    0
category_lvl3    0
Title_desc       0
dtype: int64

The missing data percent is: 0.0


In [59]:

X = train_df_imputed['Title_desc']
Y1 = train_df_imputed['category_lvl1']
Y2 = train_df_imputed['category_lvl2']
Y3 = train_df_imputed['category_lvl3']
print(Y1)
print(X)
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X_tfidf = vectorizer.transform(X)
print(X_tfidf)

0        2.0
1        3.0
2        7.0
3        3.0
4        3.0
        ... 
36278    1.0
36279    5.0
36280    1.0
36281    4.0
36282    7.0
Name: category_lvl1, Length: 36283, dtype: float64
0        adana galleri suri squar hijab light pink ul l...
1        cuba heartbreak eau de parfum spray ml oz form...
2        andoer cm cellphon smartphon mini dual head om...
3        anmyna complaint silki set shampoo ml conditio...
4        argit argiltubo green clay face bodi ml ul li ...
                               ...                        
36278    sade k led backlit wire usb mechan game keyboa...
36279    sona l electr oven seo ul li nbsp year warrant...
36280    op portabl wireless bluetooth speaker hand fre...
36281    woot woot tictacto pillow case white ul li cot...
36282    new smart wristband smart bracelet heart rate ...
Name: Title_desc, Length: 36283, dtype: object
  (0, 28913)	0.07225424844998868
  (0, 26966)	0.37659487115461027
  (0, 26075)	0.1998311858009805
  (0, 25125)

***CATEGORY 1 ***

In [60]:

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, Y1, test_size=0.3, stratify= Y1, random_state=42)
print(y_train)
print (X_train.shape)
print (X_test.shape)

SVM_Classfier=SVC(kernel='linear' , random_state=0)
model1 = SVM_Classfier.fit(X_train,y_train)
Y_Pred1=model1.predict(X_test)
print(Y_Pred1)

20539    2.0
6557     7.0
12710    3.0
12054    2.0
3503     8.0
        ... 
28094    7.0
22842    0.0
12094    2.0
25228    8.0
32451    8.0
Name: category_lvl1, Length: 25398, dtype: float64
(25398, 31798)
(10885, 31798)
[6. 7. 4. ... 5. 6. 2.]


In [61]:

print('Accuracy score with linear kernel is : ')
print(metrics.accuracy_score(y_test,Y_Pred1)*100)
# print(f' F1 Score with linear Kernel is: {f1_score(y_test, Y_Pred)*100}')
print(classification_report(y_test, Y_Pred1))
mat = confusion_matrix(y_test, Y_Pred1)
print(mat)

Accuracy score with linear kernel is : 
92.76067983463481
              precision    recall  f1-score   support

         0.0       0.96      0.91      0.93       586
         1.0       0.93      0.87      0.90       878
         2.0       0.96      0.96      0.96      1719
         3.0       0.95      0.92      0.94      1212
         4.0       0.84      0.93      0.89      1814
         5.0       0.90      0.82      0.86       475
         6.0       0.95      0.97      0.96      2183
         7.0       0.88      0.85      0.87       753
         8.0       0.97      0.95      0.96      1265

    accuracy                           0.93     10885
   macro avg       0.93      0.91      0.92     10885
weighted avg       0.93      0.93      0.93     10885

[[ 533    6    4    0   21    0   18    3    1]
 [   5  764    0    6   24    4   37   38    0]
 [   0    2 1652   11   37    1    1    1   14]
 [   0    1   18 1119   67    3    1    1    2]
 [   2   10   25   23 1691   29    9   13   1

***CATEGORY 2 ***

In [68]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, Y2, test_size=0.3, stratify= Y2, random_state=42)
print(y_train)
print (X_train.shape)
print (X_test.shape)

SVM_Classfier=SVC(kernel='linear' , random_state=0)
model2= SVM_Classfier.fit(X_train,y_train)
Y_Pred2=model2.predict(X_test)
print(Y_Pred2)

1585     46.0
23952    24.0
9668     24.0
8845      7.0
15114    47.0
         ... 
24955     0.0
27752    30.0
33144    17.0
16965    25.0
6625     34.0
Name: category_lvl2, Length: 25398, dtype: float64
(25398, 31798)
(10885, 31798)
[56. 35. 54. ...  8. 27. 35.]


In [66]:


print('Accuracy score with linear kernel is : ')
print(metrics.accuracy_score(y_test,Y_Pred2)*100)
# print(f' F1 Score with linear Kernel is: {f1_score(y_test, Y_Pred)*100}')
print(classification_report(y_test, Y_Pred2))
mat = confusion_matrix(y_test, Y_Pred2)
print(mat)




Accuracy score with linear kernel is : 
86.10932475884245
              precision    recall  f1-score   support

         0.0       0.91      0.97      0.94      2071
         1.0       0.78      0.83      0.80       350
         2.0       0.83      0.72      0.77        72
         3.0       0.64      0.69      0.67       100
         4.0       0.77      0.71      0.74       128
         5.0       0.90      0.88      0.89       107
         6.0       0.71      0.26      0.38        19
         7.0       0.86      0.84      0.85       235
         8.0       0.86      0.82      0.84       444
         9.0       0.97      0.89      0.93        36
        10.0       0.77      0.88      0.83       104
        11.0       1.00      0.75      0.86        16
        12.0       0.94      0.45      0.61        33
        13.0       0.88      0.82      0.85        17
        14.0       0.97      0.99      0.98       134
        15.0       0.92      0.85      0.89       137
        16.0       0.99

***CATEGORY 3 ***

In [71]:


X_train, X_test, y_train, y_test = train_test_split(X_tfidf, Y3, test_size=0.3, stratify= Y3, random_state=42)
print(y_train)
print (X_train.shape)
print (X_test.shape)

SVM_Classfier=SVC(kernel='linear' , random_state=0)
model3 = SVM_Classfier.fit(X_train,y_train)
Y_Pred3= model3.predict(X_test)
print(Y_Pred3)

18877     61.0
4562       1.0
28997    112.0
31983     14.0
5567      50.0
         ...  
28520     29.0
8776      31.0
34173    125.0
29604    179.0
314      125.0
Name: category_lvl3, Length: 25398, dtype: float64
(25398, 31798)
(10885, 31798)
[ 84.  61.  24. ...  31. 115. 179.]


In [72]:

print('Accuracy score with linear kernel is : ')
print(metrics.accuracy_score(y_test,Y_Pred3)*100)
# print(f' F1 Score with linear Kernel is: {f1_score(y_test, Y_Pred)*100}')
print(classification_report(y_test, Y_Pred3))
mat = confusion_matrix(y_test, Y_Pred3)
print(mat)

Accuracy score with linear kernel is : 
76.45383555351401
              precision    recall  f1-score   support

         0.0       0.59      0.62      0.61        16
         1.0       0.78      0.78      0.78       171
         2.0       0.73      0.70      0.72        27
         3.0       0.44      0.39      0.41        36
         4.0       0.83      0.59      0.69        17
         5.0       0.96      0.84      0.90        32
         6.0       0.75      0.38      0.50        16
         7.0       0.79      0.63      0.70        35
         8.0       0.47      0.38      0.42        42
         9.0       0.88      0.94      0.91       262
        10.0       0.70      0.75      0.72        51
        11.0       0.69      0.47      0.56        19
        12.0       0.79      0.51      0.62        37
        13.0       0.89      0.76      0.82        21
        14.0       0.70      0.65      0.67       133
        15.0       0.90      0.91      0.91       102
        16.0       0.67

In [27]:
le =LabelEncoder()
le.fit(["paris", "paris", "tokyo", "amsterdam"])
LabelEncoder()
list(le.classes_)
le.transform(["tokyo", "tokyo", "paris"])

list(le.inverse_transform([2]))


['tokyo']

### Rough Work


In [40]:
unique_label_c1


array(['Fashion', 'Health & Beauty',
       'TV, Audio / Video, Gaming & Wearables', 'Computers & Laptops',
       'Cameras', 'Home & Living', 'Watches Sunglasses Jewellery',
       'Mobiles & Tablets', 'Home Appliances'], dtype=object)

In [ ]:
le =LabelEncoder()
le.fit()
LabelEncoder()

### Testing

In [42]:
test_df = pd.read_csv('train_clean.csv')

In [43]:
test_df

,category_lvl1,category_lvl2,category_lvl3,Title_desc
0,2.0,56.0,113.0,adana galleri suri squar hijab light pink ul l...
1,3.0,3.0,67.0,cuba heartbreak eau de parfum spray ml oz form...
2,7.0,1.0,95.0,andoer cm cellphon smartphon mini dual head om...
3,3.0,23.0,147.0,anmyna complaint silki set shampoo ml conditio...
4,3.0,36.0,23.0,argit argiltubo green clay face bodi ml ul li ...
...,...,...,...,...
36278,1.0,8.0,82.0,sade k led backlit wire usb mechan game keyboa...
36279,5.0,30.0,109.0,sona l electr oven seo ul li nbsp year warrant...
36280,1.0,8.0,155.0,op portabl wireless bluetooth speaker hand fre...
36281,4.0,5.0,126.0,woot woot tictacto pillow case white ul li cot...


In [62]:
X = test_df['Title_desc']
v = TfidfVectorizer()
# vectorizer.fit(X)
print(X)
# X_tfidf = vectorizer.transform(X)
# print(X_tfidf)
X = v.fit_transform(test_df['Title_desc'].values.astype('U'))
print(X)

0        adana galleri suri squar hijab light pink ul l...
1        cuba heartbreak eau de parfum spray ml oz form...
2        andoer cm cellphon smartphon mini dual head om...
3        anmyna complaint silki set shampoo ml conditio...
4        argit argiltubo green clay face bodi ml ul li ...
                               ...                        
36278    sade k led backlit wire usb mechan game keyboa...
36279    sona l electr oven seo ul li nbsp year warrant...
36280    op portabl wireless bluetooth speaker hand fre...
36281    woot woot tictacto pillow case white ul li cot...
36282    new smart wristband smart bracelet heart rate ...
Name: Title_desc, Length: 36283, dtype: object
  (0, 8884)	0.20669140444780615
  (0, 6539)	0.21996803699368137
  (0, 6559)	0.16898535529139208
  (0, 13585)	0.26353252165855096
  (0, 25125)	0.09569136097916549
  (0, 4947)	0.20363684473882185
  (0, 24730)	0.2749088614258273
  (0, 24685)	0.24936730479422012
  (0, 19162)	0.1519301323425332
  (0, 16922)	

### Category 1

In [63]:
y_cat_one = model1.predict(X)

In [64]:
accuracy_score(test_df['category_lvl1'], y_cat_one)*100

95.49651351872778

### Category 2

In [69]:
y_cat_two = model2.predict(X)

In [70]:
accuracy_score(test_df['category_lvl2'], y_cat_two)*100

91.83364109913734

### Category 3

In [73]:
y_cat_three = model3.predict(X)

In [74]:
accuracy_score(test_df['category_lvl3'], y_cat_three)*100

87.10690957197585